In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import config

In [3]:
browser = webdriver.Chrome() # Mac用の記述
browser.implicitly_wait(3)

In [4]:
# SBIネオモバイルのログインページへアクセス
url_login = "https://trade.sbineomobile.co.jp/login"
browser.get(url_login)
time.sleep(3) # ページに遷移する前に次の処理が実行されないようにするため

In [7]:
# ログインフォームの要素を取得する
username = browser.find_element(By.NAME, "username")
password = browser.find_element(By.NAME, "password")
login_btn = browser.find_element(By.ID, "neo-login-btn")

# 念のためテキストボックスの中身を空にする
username.clear()
password.clear()

# テキストボックスに値を入力する
username.send_keys(config.SBI_USERNAME)
password.send_keys(config.SBI_PASSWORD)

# ログインボタンをクリックする
login_btn.click()
time.sleep(1)

In [12]:
# SBIネオモバイルのポートフォリオ
url_portfolio = "https://trade.sbineomobile.co.jp/account/portfolio"
browser.get(url_portfolio)
time.sleep(3)

In [19]:
# 銘柄一覧の要素を取得する
div_element = browser.find_element(By.CLASS_NAME, "sp-main")

# 無限スクロールが終わるまでスクロールする
last_height = browser.execute_script("return arguments[0].scrollHeight", div_element)
while True:
    browser.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", div_element)
    time.sleep(2)
    new_height = browser.execute_script("return arguments[0].scrollHeight", div_element)
    if new_height == last_height:
        break
    last_height = new_height